# Laboratorio 5: Redes Neuronales con Pytorch.    Nombre: Sandra Villca Señoranis

Se implementará una red neuronal para reconocimiento de los símbolos del alfabeto japonés Hiragana.\
Las imágenes están en el rango de valores de [0,255] en un formato csv.\
Contiene 10 Clases de 0 a 9.\
Se tiene 60000 ejemplos para entrenamiento y 10000 ejemplos para prueba.

###1. Importación de librerías para la aplicación de Pytorch

In [ ]:
# utilizado para la manipulación de directorios y rutas
import os
# Cálculo científico y vectorial para python
import numpy as np
# Libreria para graficos
from matplotlib import pyplot as plt


import torch
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.

import torchvision # torch package for vision related things
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
from tqdm import tqdm  # For nice progress bar!

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###2. Creación de la Clase RedNeuronalMLS para el manejo de la Red Neuronal Multicapa
La Clase desciende de la Clase Module.\
En el constructor tenemos dos funciones: fc1 (full connected) y fc2 (full connected).\
Se define también forward() que va a ejecutar toda la red neuronal.

In [ ]:
# Here we create our simple neural network. For more details here we are subclassing and
# inheriting from nn.Module, this is the most general way to create your networks and
# allows for more flexibility. I encourage you to also check out nn.Sequential which
# would be easier to use in this scenario but I wanted to show you something that
# "always" works.
class RedNeuronalMLS(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RedNeuronalMLS, self).__init__()
        # Our first linear layer take input_size, in this case 784 nodes to 50
        # and our second linear layer takes 50 to the num_classes we have, in
        # this case 10.
        self.fc1 = nn.Linear(input_size, 450)
        self.fc2 = nn.Linear(450, num_classes)

    def forward(self, x):
        """
        x here is the mnist images and we run it through fc1, fc2 that we created above.
        we also add a ReLU activation function in between and for that (since it has no parameters)
        I recommend using nn.functional (F)
        """
        x = self.fc1(x)
        x = F.sigmoid(x)
        # x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return x

###3. Verificar si el Colab nos da un cuda o un cpu: En éste caso nos da un cuda.
###   Asignación de los Hiperparámetros

In [ ]:
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters of our neural network which depends on the dataset, and
# also just experimenting to see what works well (learning rate for example).
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 10000
#num_epochs = 3
num_epochs = 5

cuda


###4. Creación de la Clase CustomDataset para cargar datos desde un archivo csv


In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

# Define tu clase de dataset personalizado para cargar datos desde un archivo CSV
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data.iloc[idx, 1:].values.astype('float32')  # asumiendo que las etiquetas están en la primera columna
        label = self.data.iloc[idx, 0]  # asumiendo que las etiquetas están en la primera columna
        if self.transform:
            sample = self.transform(sample)
        return sample, label

# Carga el conjunto de datos de entrenamiento y prueba
train_dataset = CustomDataset('/content/gdrive/MyDrive/SIS 420/Laboratorio5/Japanese_Characters/train_data.csv')
test_dataset = CustomDataset('/content/gdrive/MyDrive/SIS 420/Laboratorio5/Japanese_Characters/test_data.csv')

# Imprime la longitud de los conjuntos de datos
print(len(train_dataset))
print(len(test_dataset))

# Utiliza DataLoader para cargar los datos en lotes
#train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

60000
10000


###5. Creación de un modelo para la Red Neuronal

In [ ]:
# Initialize network
model = RedNeuronalMLS(input_size=input_size, num_classes=num_classes).to(device)

###En el modelo:
Primera Capa: Tiene 784 valores de entrada, 450 valores de la Capa Oculta y el bias.\
Segunda Capa: Tiene 450 valores de entrada, 10 valores de la Capa de Salida y el bias.

In [ ]:
model

RedNeuronalMLS(
  (fc1): Linear(in_features=784, out_features=450, bias=True)
  (fc2): Linear(in_features=450, out_features=10, bias=True)
)

###6. Definición de un criterio de errores
Se utiliza como criterio el CrossEntrophyLoss()\
Y como optimizador Adam().

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

###7. Entrenamiento de la red neuronal. Repetición del proceso para los epochs establecidos
scores-> son las 'y' predichas.
También se muestra el gráfico para cada iteración. En éste caso: 4 epochs

In [ ]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # for batch_idx, (data, targets) in enumerate(train_loader):

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # print(data.shape)
        # Get to correct shape
        data = data.reshape(data.shape[0], -1)
        print(data.shape)
        # print("-"*30)
        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

  0%|          | 0/6 [00:00<?, ?it/s]

torch.Size([10000, 784])


 33%|███▎      | 2/6 [00:03<00:05,  1.49s/it]

torch.Size([10000, 784])


 50%|█████     | 3/6 [00:04<00:04,  1.47s/it]

...

torch.Size([10000, 784])


 83%|████████▎ | 5/6 [00:06<00:01,  1.30s/it]

torch.Size([10000, 784])


100%|██████████| 6/6 [00:07<00:00,  1.31s/it]

torch.Size([10000, 784])


###8. Cálculo de la precisión de entrenamiento y de prueba


In [ ]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    predicciones = []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            _, predictions = scores.max(1)
            predicciones.append(predictions)

            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples, predicciones

p_train, pred_train  = check_accuracy(train_loader, model)
p_test, pred_test  = check_accuracy(test_loader, model)

print(f"Accuracy on training set: {p_train*100:.2f}")
print(f"Accuracy on test set: {p_test*100:.2f}")

Accuracy on training set: 84.25
Accuracy on test set: 69.44


###9. Observaciones: Se hizo algunas variaciones en los hiperparámetros con los siguientes resultados
PRIMER INTENTO: 50 Valores en la Capa Oculta y epochs = 3.\
Resultados: Accuracy on training set: 70.67\
Accuracy on test set: 54.62\
SEGUNDO INTENTO: 100 Valores en la Capa Oculta y epochs = 3.\
Resultados: Accuracy on training set: 73:21\
Accuracy on test set: 57:35\
TERCER INTENTO: 100 Valores en la Capa Oculta y epochs = 4.\
Resultados: Accuracy on training set: 75.59\
Accuracy on test set: 58.90\
CUARTO INTENTO: Dos capas ocultas: Capa1 con 100 Valores, Capa2 con 50 Valores, epochs = 4.\
Resultados: Accuracy on training set: 70.94\
Accuracy on test set: 56.25\
QUINTO INTENTO: 200 Valores en la Capa Oculta y epochs = 4.\
Resultados: Accuracy on training set: 79.22%\
Accuracy on test set: 63.98%\
SEXTO INTENTO: 350 Valores en la Capa Oculta y epochs = 4.\
Resultados: Accuracy on training set: 81.74%\
Accuracy on test set: 67.12%\
SÉPTIMO INTENTO: 450 Valores en la Capa Oculta y epochs = 5.\
Resultados: Accuracy on training set: 84.25%\
Accuracy on test set: 69.44%\
OCTAVO INTENTO: Dos capas ocultas: Capa1 con 450 Valores, Capa2 con 200 Valores, epochs = 4.\
Resultados: Accuracy on training set: 70.39\
Accuracy on test set: 56.98\

###10. CONCLUSIÓN
El mejor entrenamiento se consigue en el SÉPTIMO INTENTO con:\
450 Valores en la Capa Oculta y epochs = 5.\
Resultados: Accuracy on training set: 84.25%\
Accuracy on test set: 69.44%\

Éste cuadernillo contiene el código del SÉPTIMO INTENTO con los resultados mencionados.
